In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!BUILD_CUDA_EXT=0 pip install -q auto-gptq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
%%capture
!pip install accelerate peft transformers trl
!pip install datasets
!pip install transformers[torch]
!pip install tensorflow-model-optimization
#!pip install trl
#!pip install peft
#!pip install -q -U transformers peft accelerate optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/
#!pip install transformers accelerate bitsandbytes

In [ ]:
import random
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import GPT2Tokenizer, TFGPT2Model, TFGPT2ForSequenceClassification
import tensorflow as tf
import pandas as pd
from datasets import Dataset



In [ ]:
# Load quantize config, model and tokenizer
quantize_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    damp_percent=0.01,
    desc_act=False,
)

model_id = "gpt2"

#model_id = "NousResearch/Llama-2-7b-chat-hf"
out_dir = model_id + "-GPTQ"
model = AutoModelForSequenceClassification.from_pretrained(model_id, config=quantize_config, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1

In [ ]:
tokenizer_path = '/content/drive/MyDrive/tokenizer_gpt2'

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_path)

('/content/drive/MyDrive/tokenizer_gpt2/tokenizer_config.json',
 '/content/drive/MyDrive/tokenizer_gpt2/special_tokens_map.json',
 '/content/drive/MyDrive/tokenizer_gpt2/vocab.json',
 '/content/drive/MyDrive/tokenizer_gpt2/merges.txt',
 '/content/drive/MyDrive/tokenizer_gpt2/added_tokens.json',
 '/content/drive/MyDrive/tokenizer_gpt2/tokenizer.json')

In [ ]:
df = pd.read_csv("Random_Forest_Dataset.csv")

df = df.apply(pd.to_numeric, errors='coerce')

df['Anxious_or_Depressed'] = df['Anxious_or_Depressed'].map({1: 1, 2: 0})

#display(df)


# Convert the binary features and labels into a text format
def row_to_text(row):
    text = (f"Comfortable speaking about Anxiety or Depression: {'very comfortable' if row['Comfortable_speaking_about_A_or_D'] == 1 else 'somewhat' if row['Comfortable_speaking_about_A_or_D'] == 2 else 'no' if row['Comfortable_speaking_about_A_or_D'] == 3 else 'unknown'}, "
           f"Friends or Family Have Been Anxious or Depressed: {'yes' if row['Friends_Family_A_or_D'] else 'no'}, "
           f"Have Felt Anxious or Depressed More Than Once: {'yes' if row['A_or_D_more_than_once'] == 1 else 'no'}, "
           f"How Helpful Was Talking to Mental Health Professional When Anxious or Depressed: {'very helpful' if row['MH9A'] == 1 else 'somewhat' if row['MH9A'] == 2 else 'no' if row['MH9A'] == 3 else 'did not do' if row['MH9A'] == 4 else 'unknown'}, "
           f"How Helpful Was Engaging in Religious or Spiritual Activities When Anxious or Depressed: {'very helpful' if row['MH9B'] == 1 else 'somewhat' if row['MH9B'] == 2 else 'no' if row['MH9B'] == 3 else 'did not do' if row['MH9B'] == 4 else 'unknown'}, "
           f"How Helpful Was Talking to Friends or Family When Anxious or Depressed: {'very helpful' if row['MH9C'] == 1 else 'somewhat' if row['MH9C'] == 2 else 'no' if row['MH9C'] == 3 else 'did not do' if row['MH9C'] == 4 else 'unknown'}, "
           f"How Helpful Was Taking Prescribed Medication When Anxious or Depressed: {'very helpful' if row['MH9D'] == 1 else 'somewhat' if row['MH9D'] == 2 else 'no' if row['MH9D'] == 3 else 'did not do' if row['MH9D'] == 4 else 'unknown'}, "
           f"How Helpful Was Improving Healthy Lifestyle Behaviours When Anxious or Depressed: {'very helpful' if row['MH9E'] == 1 else 'somewhat' if row['MH9E'] == 2 else 'no' if row['MH9E'] == 3 else 'did not do' if row['MH9E'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Work Situation When Anxious or Depressed: {'very helpful' if row['MH9F'] == 1 else 'somewhat' if row['MH9F'] == 2 else 'no' if row['MH9F'] == 3 else 'did not do' if row['MH9F'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Personal Relationships When Anxious or Depressed: {'very helpful' if row['MH9G'] == 1 else 'somewhat' if row['MH9G'] == 2 else 'no' if row['MH9G'] == 3 else 'did not do' if row['MH9G'] == 4 else 'unknown'}, "
           f"How Helpful Was Spending Time in Nature or The Outdoors When Anxious or Depressed: {'very helpful' if row['MH9H'] == 1 else 'somewhat' if row['MH9H'] == 2 else 'no' if row['MH9H'] == 3 else 'did not do' if row['MH9H'] == 4 else 'unknown'}, "
           f"Gender: {'Male' if row['Gender'] == 1 else 'Female'}, "
           f"Household_Income: {'Poorest' if row['Household_Income'] == 1 else 'Second poorest' if row['Household_Income'] == 2 else 'Middle' if row['Household_Income'] == 3 else 'Rich' if row['Household_Income'] == 4 else 'Richest' if row['Household_Income'] == 5 else 'unknown'}, "
           f"Feelings About Household Income: {'Living comfortably' if row['Subjective_Income'] == 1 else 'Getting by' if row['Subjective_Income'] == 2 else 'Finding it difficult' if row['Subjective_Income'] == 3 else 'Finding it very difficult' if row['Subjective_Income'] == 4 else 'dont know' if row['Subjective_Income'] == 5 else 'refused' if row['Subjective_Income'] == 6 else 'unknown'}, "
           f"Employment Status: {'Full time for an employer' if row['EMP_2010'] == 1 else 'Full time for self' if row['EMP_2010'] == 2 else 'Part time' if row['EMP_2010'] == 3 else 'Unemployed' if row['EMP_2010'] == 4 else 'Part time, want full time' if row['EMP_2010'] == 5 else 'Out of workforce' if row['EMP_2010'] == 6 else 'unknown'}")
    return text


df['text'] = df.apply(row_to_text, axis=1)

display(df)

df_random_10k = df.sample(n=10000, random_state=42)
remaining_df = df.drop(df_random_10k.index)


'''class_0 = df[df['Anxious_or_Depressed'] == 0]
class_1 = df[df['Anxious_or_Depressed'] == 1]

# Sample an equal number of rows from each class
sample_size_per_class = 15000
sample_class_0 = class_0.sample(n=sample_size_per_class, random_state=42)
sample_class_1 = class_1.sample(n=sample_size_per_class, random_state=42)

# Concatenate the sampled DataFrames
first_part = pd.concat([sample_class_0, sample_class_1])

# Store the remaining samples in another DataFrame
second_part = df.drop(first_part.index)

df_dataset = Dataset.from_pandas(first_part[['text', 'Anxious_or_Depressed']])
new_df = df_dataset.to_pandas()

display(new_df)'''





,Comfortable_speaking_about_A_or_D,Friends_Family_A_or_D,Anxious_or_Depressed,A_or_D_more_than_once,MH9A,MH9B,MH9C,MH9D,MH9E,MH9F,MH9G,MH9H,Age,age_var2,Gender,Household_Income,Subjective_Income,EMP_2010,text
0,2,1,1,1,3,2,1,2,2,2,3,3,78,4,1,2,4,6,Comfortable speaking about Anxiety or Depressi...
1,2,1,1,1,2,2,2,4,4,4,2,1,63,3,2,2,4,6,Comfortable speaking about Anxiety or Depressi...
2,1,1,1,1,1,1,1,1,4,1,1,1,72,4,2,2,2,6,Comfortable speaking about Anxiety or Depressi...
3,3,1,1,1,1,4,2,1,2,2,4,1,69,4,1,5,1,6,Comfortable speaking about Anxiety or Depressi...
4,2,1,1,1,1,4,1,2,2,4,1,1,34,2,2,4,1,1,Comfortable speaking about Anxiety or Depressi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106142,2,2,0,1,3,3,2,3,3,2,3,3,40,2,1,2,1,4,Comfortable speaking about Anxiety or Depressi...
106143,3,2,0,2,4,4,2,2,2,2,3,2,30,2,1,1,4,4,Comfortable speaking about Anxiety or Depressi...
106144,3,1,0,1,4,4,3,4,2,4,3,1,19,1,2,4,1,1,Comfortable speaking about Anxiety or Depressi...
106145,3,2,0,1,4,3,2,2,3,4,2,2,48,2,2,4,3,6,Comfortable speaking about Anxiety or Depressi...


"class_0 = df[df['Anxious_or_Depressed'] == 0]\nclass_1 = df[df['Anxious_or_Depressed'] == 1]\n\n# Sample an equal number of rows from each class\nsample_size_per_class = 15000\nsample_class_0 = class_0.sample(n=sample_size_per_class, random_state=42)\nsample_class_1 = class_1.sample(n=sample_size_per_class, random_state=42)\n\n# Concatenate the sampled DataFrames\nfirst_part = pd.concat([sample_class_0, sample_class_1])\n\n# Store the remaining samples in another DataFrame\nsecond_part = df.drop(first_part.index)\n\ndf_dataset = Dataset.from_pandas(first_part[['text', 'Anxious_or_Depressed']])\nnew_df = df_dataset.to_pandas()\n\ndisplay(new_df)"

In [ ]:
# # Tokenize the dataset
# tokenized_dataset = tokenizer(df, padding=True, truncation=True, return_tensors="tf")


def tokenize_batch(batch_text):
    # Tokenize each text in the batch
    return tokenizer(batch_text, padding=True, truncation=True, max_length=128, return_tensors="pt")


tokenized_datasets = tokenize_batch(df_random_10k['text'].tolist())

tokenized_datasets['labels'] = torch.tensor(df_random_10k['Anxious_or_Depressed'].tolist())

# Convert tokenized datasets to PyTorch Dataset object
tokenized_datasets = Dataset.from_dict(tokenized_datasets)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

# Training the model
trainer.train()

# Evaluating the model
trainer.evaluate()

# Save the model
trainer.save_model("first-10k-gpt2")
trainer.save_model("/content/drive/MyDrive/first-10k-gpt2")

Epoch,Training Loss,Validation Loss
1,0.734400,0.663758
2,0.717500,0.669526
3,0.637500,0.605626


In [ ]:
'''%%time

# Quantize with GPTQ
model.quantize(
    df_dataset,
    batch_size=1,
    use_triton=True,
)



# Save model and tokenizer
model.save_quantized(out_dir, use_safetensors=True)
tokenizer.save_pretrained(out_dir)'''

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/gpt2-first-10k-for-disease-prediction-4bit")

new_text = ['Comfortable speaking about Anxiety or Depression: very comfortable, Friends or Family Have Been Anxious or Depressed: yes, Have Felt Anxious or Depressed More Than Once: yes, How Helpful Was Talking to Mental Health Professional When Anxious or Depressed: did not do, How Helpful Was Engaging in Religious or Spiritual Activities When Anxious or Depressed: did not do, How Helpful Was Talking to Friends or Family When Anxious or Depressed: very helpful, How Helpful Was Taking Prescribed Medication When Anxious or Depressed: did not do, How Helpful Was Improving Healthy Lifestyle Behaviours When Anxious or Depressed: somewhat, How Helpful Was Making a Change to Work Situation When Anxious or Depressed: did not do, How Helpful Was Making a Change to Personal Relationships When Anxious or Depressed: very helpful, How Helpful Was Spending Time in Nature or The Outdoors When Anxious or Depressed: somewhat, Gender: Female, Household_Income: Middle, Feelings About Household Income: Living comfortably, Employment Status: unknown']

encoded_inputs = tokenizer(new_text, padding=True, truncation=True, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Getting predictions
predictions = torch.argmax(outputs.logits, dim=1)

# Convert predictions to labels
predicted_labels = [model.config.id2label[p.item()] for p in predictions]

print(predicted_labels)
if predicted_labels == 'LABEL_1':
  print('The patient is likely to be anxious or depressed')
else:
  print('The patient is not likely to be anxious or depressed')


['LABEL_0']
The patient is not likely to be anxious or depressed


In [ ]:
df_100 = remaining_df.sample(n=100, random_state=42)

#df_100.head(5)

df_dataset_100 = Dataset.from_pandas(df_100[['text', 'Anxious_or_Depressed']])
new_df_100 = df_dataset_100.to_pandas()

#display(new_df_100)

from sklearn.metrics import accuracy_score
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/gpt2-first-10k-for-disease-prediction-4bit')


# Tokenize the new data
new_texts = df_100['text'].tolist()


# Initialize an empty list to store predicted labels
predicted_labels = []

# Perform inference for each input and store predictions
for text in new_texts:
    # Tokenize the input
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Getting predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Convert predictions to labels and append to list
    predicted_label = model.config.id2label[predictions[0].item()]
    predicted_labels.append(predicted_label)

label_map = {"LABEL_0": 0, "LABEL_1": 1}

# Convert predicted labels to integers
predicted_labels_int = [label_map[label] for label in predicted_labels]

# Ground truth labels
ground_truth_labels = df_100['Anxious_or_Depressed'].tolist()

# Calculate accuracy
accuracy = accuracy_score(ground_truth_labels, predicted_labels_int)

print("Accuracy of 4 bit quantised gpt2 model on test size 100:", accuracy)



Accuracy of 4 bit quantised gpt2 model on test size 100: 0.88


In [ ]:
df_1000 = remaining_df.sample(n=1000, random_state=42)

#df_100.head(5)

df_dataset_1000 = Dataset.from_pandas(df_1000[['text', 'Anxious_or_Depressed']])
new_df_1000 = df_dataset_1000.to_pandas()

#display(new_df_100)

from sklearn.metrics import accuracy_score
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/gpt2-first-10k-for-disease-prediction-4bit')


# Tokenize the new data
new_texts = df_1000['text'].tolist()


# Initialize an empty list to store predicted labels
predicted_labels = []

# Perform inference for each input and store predictions
for text in new_texts:
    # Tokenize the input
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Getting predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Convert predictions to labels and append to list
    predicted_label = model.config.id2label[predictions[0].item()]
    predicted_labels.append(predicted_label)

label_map = {"LABEL_0": 0, "LABEL_1": 1}

# Convert predicted labels to integers
predicted_labels_int = [label_map[label] for label in predicted_labels]

# Ground truth labels
ground_truth_labels = df_1000['Anxious_or_Depressed'].tolist()

# Calculate accuracy
accuracy = accuracy_score(ground_truth_labels, predicted_labels_int)

print("Accuracy of 4 bit quantised gpt2 model on test size 1000:", accuracy)



Accuracy of 4 bit quantised gpt2 model on test size 1000: 0.886


In [ ]:
df_5000 = remaining_df.sample(n=5000, random_state=42)

#df_100.head(5)

df_dataset_5000 = Dataset.from_pandas(df_5000[['text', 'Anxious_or_Depressed']])
new_df_5000 = df_dataset_5000.to_pandas()

#display(new_df_100)

from sklearn.metrics import accuracy_score
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/gpt2-first-10k-for-disease-prediction-4bit')


# Tokenize the new data
new_texts = df_5000['text'].tolist()


# Initialize an empty list to store predicted labels
predicted_labels = []

# Perform inference for each input and store predictions
for text in new_texts:
    # Tokenize the input
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Getting predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Convert predictions to labels and append to list
    predicted_label = model.config.id2label[predictions[0].item()]
    predicted_labels.append(predicted_label)

label_map = {"LABEL_0": 0, "LABEL_1": 1}

# Convert predicted labels to integers
predicted_labels_int = [label_map[label] for label in predicted_labels]

# Ground truth labels
ground_truth_labels = df_5000['Anxious_or_Depressed'].tolist()

# Calculate accuracy
accuracy = accuracy_score(ground_truth_labels, predicted_labels_int)

print("Accuracy of 4 bit quantised gpt2 model on test size 5000:", accuracy)



Accuracy of 4 bit quantised gpt2 model on test size 5000: 0.8848


In [ ]:
remaining_df.to_csv('remaining_df.csv',index=False)

In [ ]:
#next 10k - 2nd iteration

In [ ]:
tokenizer_path = "/content/drive/MyDrive/tokenizer_gpt2"

# Load the tokenizer using the model name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

def tokenize_batch(batch_text):
    # Tokenize each text in the batch
    return tokenizer(batch_text, padding=True, truncation=True, max_length=128, return_tensors="pt")


def row_to_text(row):
    text = (f"Comfortable speaking about Anxiety or Depression: {'very comfortable' if row['Comfortable_speaking_about_A_or_D'] == 1 else 'somewhat' if row['Comfortable_speaking_about_A_or_D'] == 2 else 'no' if row['Comfortable_speaking_about_A_or_D'] == 3 else 'unknown'}, "
           f"Friends or Family Have Been Anxious or Depressed: {'yes' if row['Friends_Family_A_or_D'] else 'no'}, "
           f"Have Felt Anxious or Depressed More Than Once: {'yes' if row['A_or_D_more_than_once'] == 1 else 'no'}, "
           f"How Helpful Was Talking to Mental Health Professional When Anxious or Depressed: {'very helpful' if row['MH9A'] == 1 else 'somewhat' if row['MH9A'] == 2 else 'no' if row['MH9A'] == 3 else 'did not do' if row['MH9A'] == 4 else 'unknown'}, "
           f"How Helpful Was Engaging in Religious or Spiritual Activities When Anxious or Depressed: {'very helpful' if row['MH9B'] == 1 else 'somewhat' if row['MH9B'] == 2 else 'no' if row['MH9B'] == 3 else 'did not do' if row['MH9B'] == 4 else 'unknown'}, "
           f"How Helpful Was Talking to Friends or Family When Anxious or Depressed: {'very helpful' if row['MH9C'] == 1 else 'somewhat' if row['MH9C'] == 2 else 'no' if row['MH9C'] == 3 else 'did not do' if row['MH9C'] == 4 else 'unknown'}, "
           f"How Helpful Was Taking Prescribed Medication When Anxious or Depressed: {'very helpful' if row['MH9D'] == 1 else 'somewhat' if row['MH9D'] == 2 else 'no' if row['MH9D'] == 3 else 'did not do' if row['MH9D'] == 4 else 'unknown'}, "
           f"How Helpful Was Improving Healthy Lifestyle Behaviours When Anxious or Depressed: {'very helpful' if row['MH9E'] == 1 else 'somewhat' if row['MH9E'] == 2 else 'no' if row['MH9E'] == 3 else 'did not do' if row['MH9E'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Work Situation When Anxious or Depressed: {'very helpful' if row['MH9F'] == 1 else 'somewhat' if row['MH9F'] == 2 else 'no' if row['MH9F'] == 3 else 'did not do' if row['MH9F'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Personal Relationships When Anxious or Depressed: {'very helpful' if row['MH9G'] == 1 else 'somewhat' if row['MH9G'] == 2 else 'no' if row['MH9G'] == 3 else 'did not do' if row['MH9G'] == 4 else 'unknown'}, "
           f"How Helpful Was Spending Time in Nature or The Outdoors When Anxious or Depressed: {'very helpful' if row['MH9H'] == 1 else 'somewhat' if row['MH9H'] == 2 else 'no' if row['MH9H'] == 3 else 'did not do' if row['MH9H'] == 4 else 'unknown'}, "
           f"Gender: {'Male' if row['Gender'] == 1 else 'Female'}, "
           f"Household_Income: {'Poorest' if row['Household_Income'] == 1 else 'Second poorest' if row['Household_Income'] == 2 else 'Middle' if row['Household_Income'] == 3 else 'Rich' if row['Household_Income'] == 4 else 'Richest' if row['Household_Income'] == 5 else 'unknown'}, "
           f"Feelings About Household Income: {'Living comfortably' if row['Subjective_Income'] == 1 else 'Getting by' if row['Subjective_Income'] == 2 else 'Finding it difficult' if row['Subjective_Income'] == 3 else 'Finding it very difficult' if row['Subjective_Income'] == 4 else 'dont know' if row['Subjective_Income'] == 5 else 'refused' if row['Subjective_Income'] == 6 else 'unknown'}, "
           f"Employment Status: {'Full time for an employer' if row['EMP_2010'] == 1 else 'Full time for self' if row['EMP_2010'] == 2 else 'Part time' if row['EMP_2010'] == 3 else 'Unemployed' if row['EMP_2010'] == 4 else 'Part time, want full time' if row['EMP_2010'] == 5 else 'Out of workforce' if row['EMP_2010'] == 6 else 'unknown'}")
    return text

remaining_df_90k = pd.read_csv("/content/drive/MyDrive/remaining_df_90k.csv")
remaining_df_90k['text'] = remaining_df_90k.apply(row_to_text, axis=1)

df_random_10k = remaining_df_90k.sample(n=10000, random_state=42)

remaining_df_80k = remaining_df_90k.drop(df_random_10k.index)
tokenized_datasets = tokenize_batch(df_random_10k['text'].tolist())

tokenized_datasets['labels'] = torch.tensor(df_random_10k['Anxious_or_Depressed'].tolist())

# Convert tokenized datasets to PyTorch Dataset object
tokenized_datasets = Dataset.from_dict(tokenized_datasets)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

# Training the model
trainer.train()

# Evaluating the model
trainer.evaluate()

# Save the model
trainer.save_model("gpt2-second-10k-for-disease-prediction-4bit")
trainer.save_model("/content/drive/MyDrive/gpt2-second-10k-for-disease-prediction-4bit")

remaining_df_80k.to_csv('/content/drive/MyDrive/remaining_df_80k.csv',index=False)

Epoch,Training Loss,Validation Loss
1,0.737200,0.666491
2,0.762300,0.718223
3,0.689800,0.664495


In [ ]:
df_1000 = remaining_df_80k.sample(n=1000, random_state=42)

#df_100.head(5)

df_dataset_1000 = Dataset.from_pandas(df_1000[['text', 'Anxious_or_Depressed']])
new_df_1000 = df_dataset_1000.to_pandas()

#display(new_df_100)

from sklearn.metrics import accuracy_score
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/gpt2-second-10k-for-disease-prediction-4bit')


# Tokenize the new data
new_texts = df_1000['text'].tolist()


# Initialize an empty list to store predicted labels
predicted_labels = []

# Perform inference for each input and store predictions
for text in new_texts:
    # Tokenize the input
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Getting predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Convert predictions to labels and append to list
    predicted_label = model.config.id2label[predictions[0].item()]
    predicted_labels.append(predicted_label)

label_map = {"LABEL_0": 0, "LABEL_1": 1}

# Convert predicted labels to integers
predicted_labels_int = [label_map[label] for label in predicted_labels]

# Ground truth labels
ground_truth_labels = df_1000['Anxious_or_Depressed'].tolist()

# Calculate accuracy
accuracy = accuracy_score(ground_truth_labels, predicted_labels_int)

print("Accuracy of 4 bit quantised gpt2 model on test size 1000:", accuracy)



Accuracy of 4 bit quantised gpt2 model on test size 1000: 0.904


In [ ]:
#iteration 3 -

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/gpt2-second-10k-for-disease-prediction-4bit")
tokenizer_path = "/content/drive/MyDrive/tokenizer_gpt2"

# Load the tokenizer using the model name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

def tokenize_batch(batch_text):
    # Tokenize each text in the batch
    return tokenizer(batch_text, padding=True, truncation=True, max_length=128, return_tensors="pt")


def row_to_text(row):
    text = (f"Comfortable speaking about Anxiety or Depression: {'very comfortable' if row['Comfortable_speaking_about_A_or_D'] == 1 else 'somewhat' if row['Comfortable_speaking_about_A_or_D'] == 2 else 'no' if row['Comfortable_speaking_about_A_or_D'] == 3 else 'unknown'}, "
           f"Friends or Family Have Been Anxious or Depressed: {'yes' if row['Friends_Family_A_or_D'] else 'no'}, "
           f"Have Felt Anxious or Depressed More Than Once: {'yes' if row['A_or_D_more_than_once'] == 1 else 'no'}, "
           f"How Helpful Was Talking to Mental Health Professional When Anxious or Depressed: {'very helpful' if row['MH9A'] == 1 else 'somewhat' if row['MH9A'] == 2 else 'no' if row['MH9A'] == 3 else 'did not do' if row['MH9A'] == 4 else 'unknown'}, "
           f"How Helpful Was Engaging in Religious or Spiritual Activities When Anxious or Depressed: {'very helpful' if row['MH9B'] == 1 else 'somewhat' if row['MH9B'] == 2 else 'no' if row['MH9B'] == 3 else 'did not do' if row['MH9B'] == 4 else 'unknown'}, "
           f"How Helpful Was Talking to Friends or Family When Anxious or Depressed: {'very helpful' if row['MH9C'] == 1 else 'somewhat' if row['MH9C'] == 2 else 'no' if row['MH9C'] == 3 else 'did not do' if row['MH9C'] == 4 else 'unknown'}, "
           f"How Helpful Was Taking Prescribed Medication When Anxious or Depressed: {'very helpful' if row['MH9D'] == 1 else 'somewhat' if row['MH9D'] == 2 else 'no' if row['MH9D'] == 3 else 'did not do' if row['MH9D'] == 4 else 'unknown'}, "
           f"How Helpful Was Improving Healthy Lifestyle Behaviours When Anxious or Depressed: {'very helpful' if row['MH9E'] == 1 else 'somewhat' if row['MH9E'] == 2 else 'no' if row['MH9E'] == 3 else 'did not do' if row['MH9E'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Work Situation When Anxious or Depressed: {'very helpful' if row['MH9F'] == 1 else 'somewhat' if row['MH9F'] == 2 else 'no' if row['MH9F'] == 3 else 'did not do' if row['MH9F'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Personal Relationships When Anxious or Depressed: {'very helpful' if row['MH9G'] == 1 else 'somewhat' if row['MH9G'] == 2 else 'no' if row['MH9G'] == 3 else 'did not do' if row['MH9G'] == 4 else 'unknown'}, "
           f"How Helpful Was Spending Time in Nature or The Outdoors When Anxious or Depressed: {'very helpful' if row['MH9H'] == 1 else 'somewhat' if row['MH9H'] == 2 else 'no' if row['MH9H'] == 3 else 'did not do' if row['MH9H'] == 4 else 'unknown'}, "
           f"Gender: {'Male' if row['Gender'] == 1 else 'Female'}, "
           f"Household_Income: {'Poorest' if row['Household_Income'] == 1 else 'Second poorest' if row['Household_Income'] == 2 else 'Middle' if row['Household_Income'] == 3 else 'Rich' if row['Household_Income'] == 4 else 'Richest' if row['Household_Income'] == 5 else 'unknown'}, "
           f"Feelings About Household Income: {'Living comfortably' if row['Subjective_Income'] == 1 else 'Getting by' if row['Subjective_Income'] == 2 else 'Finding it difficult' if row['Subjective_Income'] == 3 else 'Finding it very difficult' if row['Subjective_Income'] == 4 else 'dont know' if row['Subjective_Income'] == 5 else 'refused' if row['Subjective_Income'] == 6 else 'unknown'}, "
           f"Employment Status: {'Full time for an employer' if row['EMP_2010'] == 1 else 'Full time for self' if row['EMP_2010'] == 2 else 'Part time' if row['EMP_2010'] == 3 else 'Unemployed' if row['EMP_2010'] == 4 else 'Part time, want full time' if row['EMP_2010'] == 5 else 'Out of workforce' if row['EMP_2010'] == 6 else 'unknown'}")
    return text

remaining_df_80k = pd.read_csv("/content/drive/MyDrive/remaining_df_80k.csv")
remaining_df_80k['text'] = remaining_df_80k.apply(row_to_text, axis=1)

df_random_10k = remaining_df_80k.sample(n=10000, random_state=42)

remaining_df_70k = remaining_df_80k.drop(df_random_10k.index)
tokenized_datasets = tokenize_batch(df_random_10k['text'].tolist())

tokenized_datasets['labels'] = torch.tensor(df_random_10k['Anxious_or_Depressed'].tolist())

# Convert tokenized datasets to PyTorch Dataset object
tokenized_datasets = Dataset.from_dict(tokenized_datasets)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

# Training the model
trainer.train()

# Evaluating the model
trainer.evaluate()

# Save the model
trainer.save_model("gpt2-third-10k-for-disease-prediction-4bit")
trainer.save_model("/content/drive/MyDrive/gpt2-third-10k-for-disease-prediction-4bit")

remaining_df_70k.to_csv('/content/drive/MyDrive/remaining_df_70k.csv',index=False)

Epoch,Training Loss,Validation Loss
1,0.764000,0.733437
2,0.681600,0.684694
3,0.671300,0.652698


In [ ]:
df_1000 = remaining_df_70k.sample(n=1000, random_state=42)

#df_100.head(5)

df_dataset_1000 = Dataset.from_pandas(df_1000[['text', 'Anxious_or_Depressed']])
new_df_1000 = df_dataset_1000.to_pandas()

#display(new_df_100)

from sklearn.metrics import accuracy_score
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/gpt2-third-10k-for-disease-prediction-4bit')


# Tokenize the new data
new_texts = df_1000['text'].tolist()


# Initialize an empty list to store predicted labels
predicted_labels = []

# Perform inference for each input and store predictions
for text in new_texts:
    # Tokenize the input
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Getting predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Convert predictions to labels and append to list
    predicted_label = model.config.id2label[predictions[0].item()]
    predicted_labels.append(predicted_label)

label_map = {"LABEL_0": 0, "LABEL_1": 1}

# Convert predicted labels to integers
predicted_labels_int = [label_map[label] for label in predicted_labels]

# Ground truth labels
ground_truth_labels = df_1000['Anxious_or_Depressed'].tolist()

# Calculate accuracy
accuracy = accuracy_score(ground_truth_labels, predicted_labels_int)

print("Accuracy of 4 bit quantised gpt2 model on test size 1000:", accuracy)



Accuracy of 4 bit quantised gpt2 model on test size 1000: 0.895


In [ ]:
#iteration 4 -

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/gpt2-third-10k-for-disease-prediction-4bit")
tokenizer_path = "/content/drive/MyDrive/tokenizer_gpt2"

# Load the tokenizer using the model name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

def tokenize_batch(batch_text):
    # Tokenize each text in the batch
    return tokenizer(batch_text, padding=True, truncation=True, max_length=128, return_tensors="pt")


def row_to_text(row):
    text = (f"Comfortable speaking about Anxiety or Depression: {'very comfortable' if row['Comfortable_speaking_about_A_or_D'] == 1 else 'somewhat' if row['Comfortable_speaking_about_A_or_D'] == 2 else 'no' if row['Comfortable_speaking_about_A_or_D'] == 3 else 'unknown'}, "
           f"Friends or Family Have Been Anxious or Depressed: {'yes' if row['Friends_Family_A_or_D'] else 'no'}, "
           f"Have Felt Anxious or Depressed More Than Once: {'yes' if row['A_or_D_more_than_once'] == 1 else 'no'}, "
           f"How Helpful Was Talking to Mental Health Professional When Anxious or Depressed: {'very helpful' if row['MH9A'] == 1 else 'somewhat' if row['MH9A'] == 2 else 'no' if row['MH9A'] == 3 else 'did not do' if row['MH9A'] == 4 else 'unknown'}, "
           f"How Helpful Was Engaging in Religious or Spiritual Activities When Anxious or Depressed: {'very helpful' if row['MH9B'] == 1 else 'somewhat' if row['MH9B'] == 2 else 'no' if row['MH9B'] == 3 else 'did not do' if row['MH9B'] == 4 else 'unknown'}, "
           f"How Helpful Was Talking to Friends or Family When Anxious or Depressed: {'very helpful' if row['MH9C'] == 1 else 'somewhat' if row['MH9C'] == 2 else 'no' if row['MH9C'] == 3 else 'did not do' if row['MH9C'] == 4 else 'unknown'}, "
           f"How Helpful Was Taking Prescribed Medication When Anxious or Depressed: {'very helpful' if row['MH9D'] == 1 else 'somewhat' if row['MH9D'] == 2 else 'no' if row['MH9D'] == 3 else 'did not do' if row['MH9D'] == 4 else 'unknown'}, "
           f"How Helpful Was Improving Healthy Lifestyle Behaviours When Anxious or Depressed: {'very helpful' if row['MH9E'] == 1 else 'somewhat' if row['MH9E'] == 2 else 'no' if row['MH9E'] == 3 else 'did not do' if row['MH9E'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Work Situation When Anxious or Depressed: {'very helpful' if row['MH9F'] == 1 else 'somewhat' if row['MH9F'] == 2 else 'no' if row['MH9F'] == 3 else 'did not do' if row['MH9F'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Personal Relationships When Anxious or Depressed: {'very helpful' if row['MH9G'] == 1 else 'somewhat' if row['MH9G'] == 2 else 'no' if row['MH9G'] == 3 else 'did not do' if row['MH9G'] == 4 else 'unknown'}, "
           f"How Helpful Was Spending Time in Nature or The Outdoors When Anxious or Depressed: {'very helpful' if row['MH9H'] == 1 else 'somewhat' if row['MH9H'] == 2 else 'no' if row['MH9H'] == 3 else 'did not do' if row['MH9H'] == 4 else 'unknown'}, "
           f"Gender: {'Male' if row['Gender'] == 1 else 'Female'}, "
           f"Household_Income: {'Poorest' if row['Household_Income'] == 1 else 'Second poorest' if row['Household_Income'] == 2 else 'Middle' if row['Household_Income'] == 3 else 'Rich' if row['Household_Income'] == 4 else 'Richest' if row['Household_Income'] == 5 else 'unknown'}, "
           f"Feelings About Household Income: {'Living comfortably' if row['Subjective_Income'] == 1 else 'Getting by' if row['Subjective_Income'] == 2 else 'Finding it difficult' if row['Subjective_Income'] == 3 else 'Finding it very difficult' if row['Subjective_Income'] == 4 else 'dont know' if row['Subjective_Income'] == 5 else 'refused' if row['Subjective_Income'] == 6 else 'unknown'}, "
           f"Employment Status: {'Full time for an employer' if row['EMP_2010'] == 1 else 'Full time for self' if row['EMP_2010'] == 2 else 'Part time' if row['EMP_2010'] == 3 else 'Unemployed' if row['EMP_2010'] == 4 else 'Part time, want full time' if row['EMP_2010'] == 5 else 'Out of workforce' if row['EMP_2010'] == 6 else 'unknown'}")
    return text

remaining_df_70k = pd.read_csv("/content/drive/MyDrive/remaining_df_70k.csv")
remaining_df_70k['text'] = remaining_df_70k.apply(row_to_text, axis=1)

df_random_10k = remaining_df_70k.sample(n=10000, random_state=42)

remaining_df_60k = remaining_df_70k.drop(df_random_10k.index)
tokenized_datasets = tokenize_batch(df_random_10k['text'].tolist())

tokenized_datasets['labels'] = torch.tensor(df_random_10k['Anxious_or_Depressed'].tolist())

# Convert tokenized datasets to PyTorch Dataset object
tokenized_datasets = Dataset.from_dict(tokenized_datasets)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

# Training the model
trainer.train()

# Evaluating the model
trainer.evaluate()

# Save the model
trainer.save_model("gpt2-fourth-10k-for-disease-prediction-4bit")
trainer.save_model("/content/drive/MyDrive/gpt2-fourth-10k-for-disease-prediction-4bit")

remaining_df_60k.to_csv('/content/drive/MyDrive/remaining_df_60k.csv',index=False)

Epoch,Training Loss,Validation Loss
1,0.564100,0.696696
2,0.535000,0.666167
3,0.557200,0.639092


In [ ]:
remaining_df_60k.to_csv('/content/drive/MyDrive/remaining_df_60k.csv',index=False)

In [ ]:
df_1000 = remaining_df_60k.sample(n=1000, random_state=42)

#df_100.head(5)

df_dataset_1000 = Dataset.from_pandas(df_1000[['text', 'Anxious_or_Depressed']])
new_df_1000 = df_dataset_1000.to_pandas()

#display(new_df_100)

from sklearn.metrics import accuracy_score
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/gpt2-fourth-10k-for-disease-prediction-4bit')


# Tokenize the new data
new_texts = df_1000['text'].tolist()


# Initialize an empty list to store predicted labels
predicted_labels = []

# Perform inference for each input and store predictions
for text in new_texts:
    # Tokenize the input
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Getting predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Convert predictions to labels and append to list
    predicted_label = model.config.id2label[predictions[0].item()]
    predicted_labels.append(predicted_label)

label_map = {"LABEL_0": 0, "LABEL_1": 1}

# Convert predicted labels to integers
predicted_labels_int = [label_map[label] for label in predicted_labels]

# Ground truth labels
ground_truth_labels = df_1000['Anxious_or_Depressed'].tolist()

# Calculate accuracy
accuracy = accuracy_score(ground_truth_labels, predicted_labels_int)

print("Accuracy of 4 bit quantised gpt2 model on test size 1000:", accuracy)



Accuracy of 4 bit quantised gpt2 model on test size 1000: 0.9


In [ ]:
#iteration 5 -

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/gpt2-fourth-10k-for-disease-prediction-4bit")
tokenizer_path = "/content/drive/MyDrive/tokenizer_gpt2"

# Load the tokenizer using the model name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

def tokenize_batch(batch_text):
    # Tokenize each text in the batch
    return tokenizer(batch_text, padding=True, truncation=True, max_length=128, return_tensors="pt")


def row_to_text(row):
    text = (f"Comfortable speaking about Anxiety or Depression: {'very comfortable' if row['Comfortable_speaking_about_A_or_D'] == 1 else 'somewhat' if row['Comfortable_speaking_about_A_or_D'] == 2 else 'no' if row['Comfortable_speaking_about_A_or_D'] == 3 else 'unknown'}, "
           f"Friends or Family Have Been Anxious or Depressed: {'yes' if row['Friends_Family_A_or_D'] else 'no'}, "
           f"Have Felt Anxious or Depressed More Than Once: {'yes' if row['A_or_D_more_than_once'] == 1 else 'no'}, "
           f"How Helpful Was Talking to Mental Health Professional When Anxious or Depressed: {'very helpful' if row['MH9A'] == 1 else 'somewhat' if row['MH9A'] == 2 else 'no' if row['MH9A'] == 3 else 'did not do' if row['MH9A'] == 4 else 'unknown'}, "
           f"How Helpful Was Engaging in Religious or Spiritual Activities When Anxious or Depressed: {'very helpful' if row['MH9B'] == 1 else 'somewhat' if row['MH9B'] == 2 else 'no' if row['MH9B'] == 3 else 'did not do' if row['MH9B'] == 4 else 'unknown'}, "
           f"How Helpful Was Talking to Friends or Family When Anxious or Depressed: {'very helpful' if row['MH9C'] == 1 else 'somewhat' if row['MH9C'] == 2 else 'no' if row['MH9C'] == 3 else 'did not do' if row['MH9C'] == 4 else 'unknown'}, "
           f"How Helpful Was Taking Prescribed Medication When Anxious or Depressed: {'very helpful' if row['MH9D'] == 1 else 'somewhat' if row['MH9D'] == 2 else 'no' if row['MH9D'] == 3 else 'did not do' if row['MH9D'] == 4 else 'unknown'}, "
           f"How Helpful Was Improving Healthy Lifestyle Behaviours When Anxious or Depressed: {'very helpful' if row['MH9E'] == 1 else 'somewhat' if row['MH9E'] == 2 else 'no' if row['MH9E'] == 3 else 'did not do' if row['MH9E'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Work Situation When Anxious or Depressed: {'very helpful' if row['MH9F'] == 1 else 'somewhat' if row['MH9F'] == 2 else 'no' if row['MH9F'] == 3 else 'did not do' if row['MH9F'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Personal Relationships When Anxious or Depressed: {'very helpful' if row['MH9G'] == 1 else 'somewhat' if row['MH9G'] == 2 else 'no' if row['MH9G'] == 3 else 'did not do' if row['MH9G'] == 4 else 'unknown'}, "
           f"How Helpful Was Spending Time in Nature or The Outdoors When Anxious or Depressed: {'very helpful' if row['MH9H'] == 1 else 'somewhat' if row['MH9H'] == 2 else 'no' if row['MH9H'] == 3 else 'did not do' if row['MH9H'] == 4 else 'unknown'}, "
           f"Gender: {'Male' if row['Gender'] == 1 else 'Female'}, "
           f"Household_Income: {'Poorest' if row['Household_Income'] == 1 else 'Second poorest' if row['Household_Income'] == 2 else 'Middle' if row['Household_Income'] == 3 else 'Rich' if row['Household_Income'] == 4 else 'Richest' if row['Household_Income'] == 5 else 'unknown'}, "
           f"Feelings About Household Income: {'Living comfortably' if row['Subjective_Income'] == 1 else 'Getting by' if row['Subjective_Income'] == 2 else 'Finding it difficult' if row['Subjective_Income'] == 3 else 'Finding it very difficult' if row['Subjective_Income'] == 4 else 'dont know' if row['Subjective_Income'] == 5 else 'refused' if row['Subjective_Income'] == 6 else 'unknown'}, "
           f"Employment Status: {'Full time for an employer' if row['EMP_2010'] == 1 else 'Full time for self' if row['EMP_2010'] == 2 else 'Part time' if row['EMP_2010'] == 3 else 'Unemployed' if row['EMP_2010'] == 4 else 'Part time, want full time' if row['EMP_2010'] == 5 else 'Out of workforce' if row['EMP_2010'] == 6 else 'unknown'}")
    return text

remaining_df_60k = pd.read_csv("/content/drive/MyDrive/remaining_df_60k.csv")
remaining_df_60k['text'] = remaining_df_60k.apply(row_to_text, axis=1)

df_random_10k = remaining_df_60k.sample(n=10000, random_state=42)

remaining_df_50k = remaining_df_60k.drop(df_random_10k.index)
tokenized_datasets = tokenize_batch(df_random_10k['text'].tolist())

tokenized_datasets['labels'] = torch.tensor(df_random_10k['Anxious_or_Depressed'].tolist())

# Convert tokenized datasets to PyTorch Dataset object
tokenized_datasets = Dataset.from_dict(tokenized_datasets)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

# Training the model
trainer.train()

# Evaluating the model
trainer.evaluate()

# Save the model
trainer.save_model("gpt2-fifth-10k-for-disease-prediction-4bit")
trainer.save_model("/content/drive/MyDrive/gpt2-fifth-10k-for-disease-prediction-4bit")

remaining_df_50k.to_csv('/content/drive/MyDrive/remaining_df_50k.csv',index=False)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.726300,0.707067
2,0.774600,0.641641
3,0.535300,0.637129


In [ ]:
df_1000 = remaining_df_50k.sample(n=1000, random_state=42)

#df_100.head(5)

df_dataset_1000 = Dataset.from_pandas(df_1000[['text', 'Anxious_or_Depressed']])
new_df_1000 = df_dataset_1000.to_pandas()

#display(new_df_100)

from sklearn.metrics import accuracy_score
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/gpt2-fifth-10k-for-disease-prediction-4bit')


# Tokenize the new data
new_texts = df_1000['text'].tolist()


# Initialize an empty list to store predicted labels
predicted_labels = []

# Perform inference for each input and store predictions
for text in new_texts:
    # Tokenize the input
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Getting predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Convert predictions to labels and append to list
    predicted_label = model.config.id2label[predictions[0].item()]
    predicted_labels.append(predicted_label)

label_map = {"LABEL_0": 0, "LABEL_1": 1}

# Convert predicted labels to integers
predicted_labels_int = [label_map[label] for label in predicted_labels]

# Ground truth labels
ground_truth_labels = df_1000['Anxious_or_Depressed'].tolist()

# Calculate accuracy
accuracy = accuracy_score(ground_truth_labels, predicted_labels_int)

print("Accuracy of 4 bit quantised gpt2 model on test size 1000:", accuracy)



Accuracy of 4 bit quantised gpt2 model on test size 1000: 0.899


In [ ]:
#iteration 6 -

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/gpt2-fifth-10k-for-disease-prediction-4bit")
tokenizer_path = "/content/drive/MyDrive/tokenizer_gpt2"

# Load the tokenizer using the model name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

def tokenize_batch(batch_text):
    # Tokenize each text in the batch
    return tokenizer(batch_text, padding=True, truncation=True, max_length=128, return_tensors="pt")


def row_to_text(row):
    text = (f"Comfortable speaking about Anxiety or Depression: {'very comfortable' if row['Comfortable_speaking_about_A_or_D'] == 1 else 'somewhat' if row['Comfortable_speaking_about_A_or_D'] == 2 else 'no' if row['Comfortable_speaking_about_A_or_D'] == 3 else 'unknown'}, "
           f"Friends or Family Have Been Anxious or Depressed: {'yes' if row['Friends_Family_A_or_D'] else 'no'}, "
           f"Have Felt Anxious or Depressed More Than Once: {'yes' if row['A_or_D_more_than_once'] == 1 else 'no'}, "
           f"How Helpful Was Talking to Mental Health Professional When Anxious or Depressed: {'very helpful' if row['MH9A'] == 1 else 'somewhat' if row['MH9A'] == 2 else 'no' if row['MH9A'] == 3 else 'did not do' if row['MH9A'] == 4 else 'unknown'}, "
           f"How Helpful Was Engaging in Religious or Spiritual Activities When Anxious or Depressed: {'very helpful' if row['MH9B'] == 1 else 'somewhat' if row['MH9B'] == 2 else 'no' if row['MH9B'] == 3 else 'did not do' if row['MH9B'] == 4 else 'unknown'}, "
           f"How Helpful Was Talking to Friends or Family When Anxious or Depressed: {'very helpful' if row['MH9C'] == 1 else 'somewhat' if row['MH9C'] == 2 else 'no' if row['MH9C'] == 3 else 'did not do' if row['MH9C'] == 4 else 'unknown'}, "
           f"How Helpful Was Taking Prescribed Medication When Anxious or Depressed: {'very helpful' if row['MH9D'] == 1 else 'somewhat' if row['MH9D'] == 2 else 'no' if row['MH9D'] == 3 else 'did not do' if row['MH9D'] == 4 else 'unknown'}, "
           f"How Helpful Was Improving Healthy Lifestyle Behaviours When Anxious or Depressed: {'very helpful' if row['MH9E'] == 1 else 'somewhat' if row['MH9E'] == 2 else 'no' if row['MH9E'] == 3 else 'did not do' if row['MH9E'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Work Situation When Anxious or Depressed: {'very helpful' if row['MH9F'] == 1 else 'somewhat' if row['MH9F'] == 2 else 'no' if row['MH9F'] == 3 else 'did not do' if row['MH9F'] == 4 else 'unknown'}, "
           f"How Helpful Was Making a Change to Personal Relationships When Anxious or Depressed: {'very helpful' if row['MH9G'] == 1 else 'somewhat' if row['MH9G'] == 2 else 'no' if row['MH9G'] == 3 else 'did not do' if row['MH9G'] == 4 else 'unknown'}, "
           f"How Helpful Was Spending Time in Nature or The Outdoors When Anxious or Depressed: {'very helpful' if row['MH9H'] == 1 else 'somewhat' if row['MH9H'] == 2 else 'no' if row['MH9H'] == 3 else 'did not do' if row['MH9H'] == 4 else 'unknown'}, "
           f"Gender: {'Male' if row['Gender'] == 1 else 'Female'}, "
           f"Household_Income: {'Poorest' if row['Household_Income'] == 1 else 'Second poorest' if row['Household_Income'] == 2 else 'Middle' if row['Household_Income'] == 3 else 'Rich' if row['Household_Income'] == 4 else 'Richest' if row['Household_Income'] == 5 else 'unknown'}, "
           f"Feelings About Household Income: {'Living comfortably' if row['Subjective_Income'] == 1 else 'Getting by' if row['Subjective_Income'] == 2 else 'Finding it difficult' if row['Subjective_Income'] == 3 else 'Finding it very difficult' if row['Subjective_Income'] == 4 else 'dont know' if row['Subjective_Income'] == 5 else 'refused' if row['Subjective_Income'] == 6 else 'unknown'}, "
           f"Employment Status: {'Full time for an employer' if row['EMP_2010'] == 1 else 'Full time for self' if row['EMP_2010'] == 2 else 'Part time' if row['EMP_2010'] == 3 else 'Unemployed' if row['EMP_2010'] == 4 else 'Part time, want full time' if row['EMP_2010'] == 5 else 'Out of workforce' if row['EMP_2010'] == 6 else 'unknown'}")
    return text

remaining_df_50k = pd.read_csv("/content/drive/MyDrive/remaining_df_50k.csv")
remaining_df_50k['text'] = remaining_df_50k.apply(row_to_text, axis=1)

df_random_10k = remaining_df_50k.sample(n=10000, random_state=42)

remaining_df_40k = remaining_df_50k.drop(df_random_10k.index)
tokenized_datasets = tokenize_batch(df_random_10k['text'].tolist())

tokenized_datasets['labels'] = torch.tensor(df_random_10k['Anxious_or_Depressed'].tolist())

# Convert tokenized datasets to PyTorch Dataset object
tokenized_datasets = Dataset.from_dict(tokenized_datasets)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

# Training the model
trainer.train()

# Evaluating the model
trainer.evaluate()

# Save the model
trainer.save_model("gpt2-sixth-10k-for-disease-prediction-4bit")
trainer.save_model("/content/drive/MyDrive/gpt2-sixth-10k-for-disease-prediction-4bit")

remaining_df_40k.to_csv('/content/drive/MyDrive/remaining_df_40k.csv',index=False)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.609400,0.695003
2,0.534500,0.682979
3,0.622800,0.684122


In [ ]:
remaining_df_40k = pd.read_csv("/content/drive/MyDrive/remaining_df_40k.csv")
tokenizer_path = "/content/drive/MyDrive/tokenizer_gpt2"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
df_1000 = remaining_df_40k.sample(n=1000, random_state=42)

#df_100.head(5)

df_dataset_1000 = Dataset.from_pandas(df_1000[['text', 'Anxious_or_Depressed']])
new_df_1000 = df_dataset_1000.to_pandas()

#display(new_df_100)

from sklearn.metrics import accuracy_score
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/gpt2-sixth-10k-for-disease-prediction-4bit')


# Tokenize the new data
new_texts = df_1000['text'].tolist()


# Initialize an empty list to store predicted labels
predicted_labels = []

# Perform inference for each input and store predictions
for text in new_texts:
    # Tokenize the input
    encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Getting predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Convert predictions to labels and append to list
    predicted_label = model.config.id2label[predictions[0].item()]
    predicted_labels.append(predicted_label)

label_map = {"LABEL_0": 0, "LABEL_1": 1}

# Convert predicted labels to integers
predicted_labels_int = [label_map[label] for label in predicted_labels]

# Ground truth labels
ground_truth_labels = df_1000['Anxious_or_Depressed'].tolist()

# Calculate accuracy
accuracy = accuracy_score(ground_truth_labels, predicted_labels_int)

print("Accuracy of 4 bit quantised gpt2 model on test size 1000:", accuracy)



Accuracy of 4 bit quantised gpt2 model on test size 1000: 0.898


In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')

import torch
import numpy as np
import pandas as pd
import os
from transformers import AutoConfig, AutoModelForSequenceClassification

# Load the test dataset from a CSV file
test_df = pd.read_csv('/content/drive/MyDrive/remaining_df_40k.csv')

# Remove the 'text' feature
test_df.drop('text', axis=1, inplace=True)

# Load the trained model from your Google Drive
model_path = '/content/drive/MyDrive/gpt2-sixth-10k-for-disease-prediction-4bit'
config = AutoConfig.from_pretrained(model_path + '/config.json')
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

# Set the model to evaluation mode
model.eval()

# Convert features to tensor
features_tensor = torch.tensor(test_df.values, dtype=torch.long)

# Make predictions
with torch.no_grad():
    outputs = model(input_ids=features_tensor)

# Convert outputs to probabilities
probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)

# Assuming the first class is 'no mental illness' and the second class is 'mental illness'
predictions = np.argmax(probabilities.numpy(), axis=1)

# Add predictions to the test dataset
test_df['predictions'] = predictions

# Save the updated test dataset with predictions
test_df.to_csv('/content/lol', index=False)
'''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Mounted at /content/drive
